In [ ]:


import os
os.chdir(r'C:\code\bedford-ubid')

import plotly.io as pio
import contextily as ctx  # For basemaps
from scipy import stats
from pathlib import Path

from urllib.parse import urlencode

import plotly.graph_objects as go
import plotly.express as px
from folium import plugins
import folium
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import numpy as np
import warnings
import pyproj
from pyproj import CRS
import xlwings as xw
from pymodule.folium_plots import create_folium_polygon

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")


# Set the default template to "plotly_white"

pio.templates.default = "simple_white"
%load_ext autoreload
%autoreload 2

In [ ]:
# building_parcel_match= pd.read_pickle('./dataprocess/building_parcel_match.pickle')
# xlbook = xw.Book('./datasets/Multifamily Building Stock.xlsx')

# missing_parcels = pd.read_pickle('./dataprocess/missing_parcels.pickle')



In [ ]:
cbl = pd.DataFrame(xlbook.sheets['Sheet1'].used_range.value)
cbl.columns = cbl.iloc[0,:]
cbl = cbl.iloc[1:,:]
cbl

missing_parcels = [x for x in cbl['Parcel ID'].unique() if x not in building_parcel_match['PRINT_KEY'].unique()]

len(missing_parcels)
len(cbl)

122

In [ ]:
cbl_match = building_parcel_match.loc[building_parcel_match['PRINT_KEY'].isin(cbl['Parcel ID'])]

cbl_merge = pd.merge(
    left=cbl,
    right=building_parcel_match,
    left_on='Parcel ID',
    right_on='PRINT_KEY',
    how='left'
)

buildings_per_parcel = cbl_merge['Parcel ID'].value_counts()

fig = px.histogram(buildings_per_parcel.values)

fig.update_layout(
    title='distribution, buildings per parcel - BEDFORD',
    width=600, height=400
)
fig.show()



cbl_merge.loc[cbl_merge['Parcel ID'] == '49.15-4-31']

,Parcel ID,Address,Town,Zip Codes,# of Units,geometry,meanHeight,RegionName,quadkey,building_idx,...,SWIS_SBL_I,SWIS_PRINT,ROLL_YR,SPATIAL_YR,OWNER_TYPE,NYS_NAME,NYS_NAME_S,DUP_GEO,CALC_ACRES,geometry_parcel
32,49.15-4-31,107-109 Katonah Ave,Katonah,10536.0,2.0,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


### parcels not found in westchester tax database (may need extra processing)

In [21]:
missing_parcels
cbl.loc[cbl['Parcel ID'].isin(missing_parcels)]


,Parcel ID,Address,Town,Zip Codes,# of Units
27,49.15-4-31,107-109 Katonah Ave,Katonah,10536.0,2.0
36,49.10-3-8,80 Greenville Rd,Katonah,10536.0,3.0
46,49.11-1-17,20 Fisher Ln,Katonah,10536.0,2.0
56,49.15-4-19,17-19 Parkway,Katonah,10536.0,2.0
57,60.13-1-30,Haines Rd,Bedford,10506.0,64.0
58,60.14-3-49,132 Babbitt Rd,Bedford,10507.0,2.0
64,60.14-4-9,141 Babbitt Rd,Bedford Hills,10507.0,2.0
73,49.15-4-49,18-24 Valley Rd,Katonah,10536.0,3.0
75,49.15-4-49,18-24 Valley Rd,Katonah,10536.0,2.0
103,49.15-4-22,26-32 Parkway,Katonah,10536.0,3 or 4


### set up / reformat

In [ ]:
## create duplicate of cbl_merge (one for parcel outline, one for building outline) -- assigning crs requires single 'geometry' column
parcel_geo = cbl_merge.copy()
parcel_geo['geometry'] = parcel_geo['geometry_parcel']
parcel_geo = gpd.GeoDataFrame(parcel_geo.drop('geometry_parcel', axis=1), crs='EPSG:2262')


building_geo = cbl_merge.copy().drop('geometry_parcel', axis=1)
building_geo = gpd.GeoDataFrame(building_geo, crs='EPSG:2262')


parcel_geo = parcel_geo.to_crs('EPSG:4326')
building_geo = building_geo.to_crs('EPSG:4326')

parcel_geo['label'] = 'parcel'
building_geo['label'] = 'building'



In [ ]:

"""MAP SETUP"""
center_lat = parcel_geo.geometry.centroid.y.iloc[0]
center_lon = parcel_geo.geometry.centroid.x.iloc[0]


fmap = folium.Map(location=[center_lat, center_lon], zoom_start=15)

meta_fields = ['label', 'PARCEL_ADD', 'MAIL_ADDR', 'PRINT_KEY']
meta_aliases = meta_fields


"""CREATE BUILDING POLYGONS"""
building_polygons = folium.GeoJson(
    building_geo,
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.2,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=meta_fields,
        aliases=meta_aliases,
        sticky=False,
        labels=True
    ),
    popup=folium.GeoJsonPopup(
        fields=meta_fields,
        aliases=meta_aliases,
        labels=True
    ),
    highlight_function=lambda x: {
        'fillOpacity': 0.8,  # Hover opacity
    },
)


"""CREATE PARCEL POLYGONS"""
parcel_polygons = folium.GeoJson(
    parcel_geo,
    style_function=lambda feature: {
        'fillColor': 'green',
        'color': 'green',
        'weight': 3,
        'fillOpacity': 0.1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=meta_fields,
        aliases=meta_aliases,
        sticky=False,
        labels=True
    ),
    popup=folium.GeoJsonPopup(
        fields=meta_fields,
        aliases=meta_aliases,
        labels=True
    ),
    highlight_function=lambda x: {
        'fillOpacity': 0.8,  # Hover opacity
    },
)


"""OUTPUT DATA"""

parcel_polygons.add_to(fmap)
building_polygons.add_to(fmap)


fmap.save('html_out/polygon_map.html')

In [24]:

# parcel_coords = parcels_proj.loc[outlier_buildings.index].geometry


# gdf_parcel = gpd.GeoDataFrame(parcel_coords, crs='EPSG:2262')
# gdf_parcel_web = gdf_parcel.to_crs('EPSG:4326')


# """ESTABLISH MAP"""
# center_lat = gdf_parcel_web.geometry.centroid.y.iloc[0]
# center_lon = gdf_parcel_web.geometry.centroid.x.iloc[0]

# m = folium.Map(location=[center_lat, center_lon], zoom_start=15)


# """BUILDING OUTLINES"""
# buildings_within_outliers = building_parcel_match.loc[building_parcel_match.parcel_idx.isin(outlier_buildings.index)]
# building_coordinates = buildings_within_outliers.geometry

# gdf_building = gpd.GeoDataFrame(buildings_within_outliers, crs='EPSG:2262')
# gdf_building_web = gdf_building.to_crs('EPSG:4326')
# # set to string for json processing
# gdf_building_web['geometry_parcel'] = gdf_building_web['geometry_parcel'].astype(str)

# fields = ['PARCEL_ADD', 'MAIL_ADDR', 'PRINT_KEY']

# # for row in gdf_building_web['geometry']:
# folium.GeoJson(
#     gdf_building_web.__geo_interface__,
#     style_function=lambda feature: {
#         'fillColor': 'blue',
#         'color': 'red',
#         'weight': 2,
#         'fillOpacity': 0.0,
#     },
#     tooltip=folium.GeoJsonTooltip(
#         fields=fields,
#         aliases=fields,
#         sticky=False,
#         labels=True
#     ),
#     popup=folium.GeoJsonPopup(
#         fields=fields,
#         aliases=fields,
#         labels=True
#     )
# ).add_to(m)

# m.save('polygon_map.html')